# Initialization

In [1]:
from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
# import imgaug as ia
# from tqdm import tqdm
# from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2
# from preprocessing import parse_annotation, BatchGenerator
from utils import WeightReader, decode_netout #, draw_boxes

from glob import glob

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%matplotlib inline

Using TensorFlow backend.


In [2]:
# LABELS = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# LABELS = ['Fixed-wing Aircraft', 'Small Aircraft', 'Cargo Plane', 'Helicopter', 'Passenger Vehicle', 'Small Car', 'Bus', 'Pickup Truck', 'Utility Truck', 'Truck', 'Cargo Truck', 'Truck w/Box', 'Truck Tractor', 'Trailer', 'Truck w/Flatbed', 'Truck w/Liquid', 'Crane Truck', 'Railway Vehicle', 'Passenger Car', 'Cargo Car', 'Flat Car', 'Tank car', 'Locomotive', 'Maritime Vessel', 'Motorboat', 'Sailboat', 'Tugboat', 'Barge', 'Fishing Vessel', 'Ferry', 'Yacht', 'Container Ship', 'Oil Tanker', 'Engineering Vehicle', 'Tower crane', 'Container Crane', 'Reach Stacker', 'Straddle Carrier', 'Mobile Crane', 'Dump Truck', 'Haul Truck', 'Scraper/Tractor', 'Front loader/Bulldozer', 'Excavator', 'Cement Mixer', 'Ground Grader', 'Hut/Tent', 'Shed', 'Building', 'Aircraft Hangar', 'Damaged Building', 'Facility', 'Construction Site', 'Vehicle Lot', 'Helipad', 'Storage Tank', 'Shipping container lot', 'Shipping Container', 'Pylon', 'Tower']
#LABELS = ['Small Car','Truck','Bus']
LABELS=['Building']
IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13, 13   # Vince: changed from 13 to 8
BOX              = 5
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.5#0.5
NMS_THRESHOLD    = 0.2#0.45
#ANCHORS          = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
ANCHORS          = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

# Construct the network

In [8]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [9]:
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3))
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4))

# Layer 1
x = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
x = BatchNormalization(name='norm_1')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 2
x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(x)
x = BatchNormalization(name='norm_2')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 3
x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(x)
x = BatchNormalization(name='norm_3')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 4
x = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(x)
x = BatchNormalization(name='norm_4')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 5
x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(x)
x = BatchNormalization(name='norm_5')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 6
x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
x = BatchNormalization(name='norm_6')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 7
x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(x)
x = BatchNormalization(name='norm_7')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 8
x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(x)
x = BatchNormalization(name='norm_8')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 9
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(x)
x = BatchNormalization(name='norm_9')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 10
x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(x)
x = BatchNormalization(name='norm_10')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 11
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(x)
x = BatchNormalization(name='norm_11')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 12
x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(x)
x = BatchNormalization(name='norm_12')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 13
x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(x)
x = BatchNormalization(name='norm_13')(x)
x = LeakyReLU(alpha=0.1)(x)

skip_connection = x

x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 14
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(x)
x = BatchNormalization(name='norm_14')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 15
x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(x)
x = BatchNormalization(name='norm_15')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 16
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(x)
x = BatchNormalization(name='norm_16')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 17
x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(x)
x = BatchNormalization(name='norm_17')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 18
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(x)
x = BatchNormalization(name='norm_18')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 19
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(x)
x = BatchNormalization(name='norm_19')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 20
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x)
x = BatchNormalization(name='norm_20')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

x = concatenate([skip_connection, x])

# Layer 22
x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(x)
x = BatchNormalization(name='norm_22')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 23
x = Conv2D(BOX * (4 + 1 + CLASS), (1,1), strides=(1,1), padding='same', name='conv_23')(x)
output = Reshape((GRID_H, GRID_W, BOX, 4 + 1 + CLASS))(x)

# small hack to allow true_boxes to be registered when Keras build the model 
# for more information: https://github.com/fchollet/keras/issues/2790
output = Lambda(lambda args: args[0])([output, true_boxes])

model = Model([input_image, true_boxes], output)

In [10]:
# model.summary()

**Parse the annotations to construct train generator and validation generator**

In [11]:
generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'BOX'             : BOX,
    'LABELS'          : LABELS,
    'CLASS'           : len(LABELS),
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : 50,
}

# Perform detection on image

In [7]:
model.load_weights("weights_building.h5")

ValueError: Dimension 0 in both shapes must be equal, but are 1 and 30 for 'Assign_110' (op: 'Assign') with input shapes: [1,1,1024,40], [30,1024,1,1].

In [ ]:
from hyper import IMG_ID as img_id
# from hyper import NUM_SQ
import gc_util as gc

sqs = [216, 217, 218]
# sqs = [i for i in range(1, NUM_SQ+1)]
gc.open_images(img_id, sqs)
    
val_files = ['LSMS_dg/dg_lsms_uganda_1000x1000_%s_%s.jpeg' % (img_id, sq) for sq in sqs]
# val_files = glob(img_files)

In [ ]:
def fn(name):
    return int(name[42:-5])  # parsing out the sq variable
    
val_files = sorted(val_files, key=fn)
# val_files = val_files[:NUM_SQ]
# for file in val_files: print(file)

In [12]:
print(len(val_files))

3


In [28]:
from hyper import WH
COORDS = [(i, j) for i in range(0,1000,WH) for j in range(0,1000,WH)]

In [27]:
# from hyper import OBJ_THRESH as obj_thresh
obj_thresh = 0.25

In [29]:
import time
NUM_SQ = len(val_files)

def object_detection(val_files):
    print("Performing object detection on %s tiles, each %s chips of w,h=%s" % (NUM_SQ, int((1000/WH)**2), WH))
    print()
    
    start = time.time()
    for k, img_path in enumerate(val_files):
        # check if file has already been scanned
        
        tst_name = 'boxes/' + img_path[8:-5] + '_' + str(int((1000/WH)**2) - 1) + '.pickle'
        if os.path.isfile(tst_name):
            continue
        
        if (k+1) % 10 == 0:
            pdone = float(k) / len(val_files)
            curr_time = time.time()
            rem = int((1-pdone)/pdone * (curr_time - start))
            print("%.2f percent done at sq %s. ETC: %s:%02s" % (100 * pdone, k+1, int(rem / 60), rem % 60))

        image = cv2.imread(img_path)
        # no longer resizing image here

        for idx, coord_pair in enumerate(COORDS):
            image_quarter = image[coord_pair[0]:coord_pair[0]+WH,
                                  coord_pair[1]:coord_pair[1]+WH]

            dummy_array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))
            input_image = cv2.resize(image_quarter, (416, 416))
            input_image = input_image / 255.
            input_image = input_image[:,:,::-1]
            input_image = np.expand_dims(input_image, 0)
            netout = model.predict([input_image, dummy_array])

            boxes = decode_netout(netout[0],
                              obj_threshold=obj_thresh,
                              nms_threshold=NMS_THRESHOLD,
                              anchors=ANCHORS,
                              nb_class=CLASS)

            for box in boxes:
                image_h, image_w, _ = image_quarter.shape

                box.xmin *= image_w
                box.ymin *= image_h
                box.xmax *= image_w
                box.ymax *= image_h

            out_name = 'boxes/' + img_path[8:-5] + '_' + str(idx) + '.pickle'
            pickle_out = open(out_name, "wb")
            pickle.dump(boxes, pickle_out)
            pickle_out.close()
            
    print()
    print("Object detection completed!")

In [32]:
object_detection(val_files)

Performing object detection on 3 tiles, each 4 chips of w,h=500


Object detection completed!


In [33]:
from hyper import HOUSE_SIZE_THRESH
from hyper import HOUSE_LEN_THRESH

def draw_boxes_coords(img_id, sq, coords=COORDS):
    img_path = "LSMS_dg/dg_lsms_uganda_1000x1000_" + str(img_id) + "_" + str(sq) + ".jpeg"
    
    imgs = []
    
    for idx, coord_pair in enumerate(coords):
        pickle_path = "boxes/dg_lsms_uganda_1000x1000_" + str(img_id) + "_" + str(sq) + "_" + str(idx) + ".pickle"

        pickle_in = open(pickle_path, "rb")
        boxes = pickle.load(pickle_in)

        img = cv2.imread(img_path)
        img = img[coord_pair[0]:coord_pair[0]+WH,
                  coord_pair[1]:coord_pair[1]+WH]

        for tbox in boxes:
            size = (tbox.xmax - tbox.xmin) * (tbox.ymax - tbox.ymin)
            if size > HOUSE_SIZE_THRESH or (tbox.xmax - tbox.xmin) > HOUSE_LEN_THRESH or (tbox.ymax - tbox.ymin) > HOUSE_LEN_THRESH: continue
            
            p1 = (int(tbox.xmin), int(tbox.ymin))
            p2 = (int(tbox.xmax), int(tbox.ymax))

            img = cv2.rectangle(img, p1, p2, (255,0,0), thickness=1)
    
        imgs.append(img)
    
    return imgs

In [34]:
def save_boxes_coords(img_id, sq, coords=COORDS):
    imgs = draw_boxes_coords(img_id, sq)
    
    if WH == 250:
        rws = []
        for k in range(0,16,4):
            rws.append(np.concatenate([imgs[k], imgs[k+1], imgs[k+2], imgs[k+3]], axis=1))
        img = np.concatenate([rws[0], rws[1], rws[2], rws[3]], axis=0)
        path = "box_imgs/dg_lsms_uganda_1000x1000_%s_%s_boxed.jpeg" % (img_id, sq)
        cv2.imwrite(path, img)
    
    elif WH == 500:
        for idx in range(len(coords)):
            path = "box_imgs/dg_lsms_uganda_1000x1000_" + str(img_id) + "_" + str(sq) + "_" + str(idx) + ".jpeg"
            cv2.imwrite(path, imgs[idx])

In [35]:
for sq in sqs:
    save_boxes_coords(img_id, sq)

# Delete folders for restart

In [18]:
def empty_folder(folder):
    import shutil
    if os.path.exists(folder):
        shutil.rmtree(folder)
        os.makedirs(folder)

In [19]:
empty_folder("box_imgs")

In [20]:
empty_folder("boxes")